## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## 2. Reading Data

In [ ]:
df = pd.read_csv('../input/sonar-dataset-suitable-for-classification/sonar.all-data.csv')

## 3. Data overview

In [ ]:
df.head()

In [ ]:
df.info()

### Fortunately we got 60 clean features!

In [ ]:
df.describe()

In [ ]:
df["Label"].describe()

### Label consists of two values, M and R.

### Because all of the features are the same type, EDA won't give us any intuiton. so let's just move to model training!

## 4. Determining features and label

In [ ]:
X = df.drop(["Label"], axis = 1)
y = df["Label"]

## 5. Spliting the test and train data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

## 6. Scaling the features for pipeline

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

## 7. Creating the model for pipeling

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

## 8. Building the pipeline

In [ ]:
operations = [("Scaler",scaler),("KNN", knn)]

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(operations)

## 9. Finding best K with Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

k_values = list(range(1,20))

In [ ]:
pipeline.get_params().keys()

In [ ]:
param_grid = {"KNN__n_neighbors" : k_values}

In [ ]:
cv_classifier = GridSearchCV(pipeline,param_grid=param_grid,scoring="accuracy")

In [ ]:
cv_classifier.fit(X_train,y_train)

In [ ]:
cv_classifier.best_estimator_.get_params()

### Best K = 1 via Grid Search

## 10. Finding best K with Elbow method

In [ ]:
scaler.fit(X_train)
scaled_X_train= scaler.transform(X_train)
scaled_X_test= scaler.transform(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
error = []

for i in range(1,40):
    knn_model = KNeighborsClassifier(n_neighbors=i)
    knn_model.fit(scaled_X_train,y_train)
    
    y_pred = knn_model.predict(scaled_X_test)
    
    e = 1- accuracy_score(y_test, y_pred)
    error.append(e)

error

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, 40), error, label='Test Error')
plt.legend()
plt.ylabel('Error Rate')
plt.xlabel('K Value')

### Best k = 1 via Elbow method

## 11. Evaluating the final model

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model.fit(scaled_X_train,y_train)
y_pred = knn_model.predict(scaled_X_test)

In [ ]:
y_pred

In [ ]:
print(classification_report(y_test, y_pred))

### 83% Accuracy!